Rossman Stores Sales Project by Group 7
=======

Rossmann Store Sales--Forecast sales using store, promotion, and competitor data

Problem
=======

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.The task is to forecast the "Sales" column for the test set.

Solution
=======
Performing following set of tasks on the dataset: 

a. Data exploration including data aggregation and summarization 
b. Tune hyperparameters 
c. Create prediction model
d. Forecast sales

In [1]:
# Call data manipulation libraries
import numpy as np
import pandas as pd
import seaborn as sns
#Graphing
import matplotlib.pyplot as plt 

#Data transformation classes
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#Dimensionality reduction
from sklearn.decomposition import PCA 

#Data splitting and model parameter search
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from xgboost.sklearn import XGBRegressor
#Model pipelining
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Random forest modelling
from sklearn.ensemble import RandomForestRegressor
import time
import gc
from scipy.stats import uniform
import calendar

#Display all outputs of one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#To read data of train and store file
ross_train = pd.read_csv("train.csv",low_memory=False)
ross_store=store = pd.read_csv("store.csv")
ross_train.head()
ross_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [3]:
#To check for null value in the datasets
ross_train.isnull().sum()
ross_store.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [4]:
#Merging train and store file
data = pd.merge(ross_train, ross_store,on = 'Store', how='left')
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
data.shape


(1017209, 18)

In [6]:
#Considering only the records have Sales > 0 since the dataset is already too large
#data = data[data['Sales'] > 0]

data.dropna(inplace = True)

In [7]:
#to check the reduction in the dataset
data.shape

(324326, 18)

In [8]:
def checkpromomonth(row):
 if (row['MonthName'] in row['PromoInterval']):
    return 1
 else:
    return 0

In [9]:
def ProcessData(data):
    data["CompetitionDistance"].fillna(data["CompetitionDistance"].mean(), inplace = True)
    
    data['StateHoliday']= data['StateHoliday'].map({'0':0, 0: 0,'a':1, 'b' : 2, 'c': 3})
    
    data['Date']=pd.to_datetime(data['Date'])
    data['Year']=data['Date'].dt.year
    data['MonthNumber']=data['Date'].dt.month
    data['MonthName']=data['MonthNumber'].apply(lambda x: calendar.month_abbr[x])
    data['Day']=data['Date'].dt.day
    data['WeekNumber']=data['Date'].dt.weekofyear

    data['CompetitionOpen'] = 12 * (data['Year'] - data['CompetitionOpenSinceYear']) + (data['MonthNumber'] - data['CompetitionOpenSinceMonth'])
    data['CompetitionOpen'] = data['CompetitionOpen'].apply(lambda x: x if x > 0 else 0)

    data['Promo2Open'] = 12 * (data['Year'] - data['Promo2SinceYear']) + (data['WeekNumber'] - data['Promo2SinceWeek']) / float(4)
    data['Promo2Open'] = data['Promo2Open'].apply(lambda x: x if x > 0 else 0)

    data['PromoInterval']=data['PromoInterval'].astype(str)
    
    data['IsPromoMonth'] =  data.apply(lambda row: checkpromomonth(row),axis=1)

    data.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'], axis = 1,  inplace = True)
    data.drop(['Promo2SinceYear', 'Promo2SinceWeek'], axis = 1,  inplace = True)
    data.drop(['Date', 'MonthName','PromoInterval'], axis = 1,  inplace = True)


In [10]:
ProcessData(data)

<ipython-input-9-578d5171619f>:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['WeekNumber']=data['Date'].dt.weekofyear


In [11]:
#To check for null value in the dataset
data.isnull().sum()
data.shape
data.head()

Store                  0
DayOfWeek              0
Sales                  0
Customers              0
Open                   0
Promo                  0
StateHoliday           0
SchoolHoliday          0
StoreType              0
Assortment             0
CompetitionDistance    0
Promo2                 0
Year                   0
MonthNumber            0
Day                    0
WeekNumber             0
CompetitionOpen        0
Promo2Open             0
IsPromoMonth           0
dtype: int64

(324326, 19)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,MonthNumber,Day,WeekNumber,CompetitionOpen,Promo2Open,IsPromoMonth
1,2,5,6064,625,1,1,0,1,a,a,570.0,1,2015,7,31,31,92.0,64.50,1
2,3,5,8314,821,1,1,0,1,a,a,14130.0,1,2015,7,31,31,103.0,52.25,1
10,11,5,10457,1236,1,1,0,1,a,c,960.0,1,2015,7,31,31,44.0,43.50,1
13,14,5,6544,710,1,1,0,1,a,a,1300.0,1,2015,7,31,31,16.0,45.75,1
14,15,5,9191,766,1,1,0,1,d,c,4110.0,1,2015,7,31,31,64.0,52.25,1


In [12]:
data['StoreType'].value_counts()
data['Assortment'].value_counts()
data['StoreType']= data['StoreType'].map({'a':1, 'b' : 2, 'c': 3, 'd' : 4})
data['Assortment'] = data['Assortment'].map({'a':1, 'b' : 2, 'c': 3})

a    177912
d    103796
c     42618
Name: StoreType, dtype: int64

a    189304
c    135022
Name: Assortment, dtype: int64

In [13]:
data = data.astype('int32')

In [14]:
y = data['Sales']
data.drop(['Sales','Customers'], axis = 1,  inplace = True)

In [15]:
num_columns = data.columns[data.nunique() > 12]
cat_columns = data.columns[data.nunique() <= 12]
num_columns
cat_columns

Index(['Store', 'CompetitionDistance', 'Day', 'WeekNumber', 'CompetitionOpen',
       'Promo2Open'],
      dtype='object')

Index(['DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'Promo2', 'Year', 'MonthNumber',
       'IsPromoMonth'],
      dtype='object')

In [16]:
ct=ColumnTransformer([
    ('rs',RobustScaler(),num_columns),
    ('ohe',OneHotEncoder(),cat_columns),
    ],
    remainder="passthrough"
    )
ct.fit_transform(data)

array([[-0.98076923, -0.33001988,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.97902098,  2.36580517,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.96503497, -0.25248509,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.95104895, -0.16500994, -1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.95454545,  0.25049702, -1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.95804196, -0.06560636, -1.        , ...,  0.        ,
         0.        ,  1.        ]])

In [17]:
X=data

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.30)
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(227028, 17)

(97298, 17)

(227028,)

(97298,)

In [19]:
steps_xg = [('sts', StandardScaler() ),
            ('pca', PCA()),
            ('xg',  XGBRegressor(objective='reg:squarederror',silent = False, n_jobs=3, reg_lambda=1,gamma=0))
            ]

pipe_xg = Pipeline(steps_xg)

pipe_xg.get_params()

{'memory': None,
 'steps': [('sts', StandardScaler()),
  ('pca', PCA()),
  ('xg',
   XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=None, gamma=0, gpu_id=None,
                importance_type='gain', interaction_constraints=None,
                learning_rate=None, max_delta_step=None, max_depth=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=3, num_parallel_tree=None,
                random_state=None, reg_alpha=None, reg_lambda=1,
                scale_pos_weight=None, silent=False, subsample=None,
                tree_method=None, validate_parameters=None, verbosity=None))],
 'verbose': False,
 'sts': StandardScaler(),
 'pca': PCA(),
 'xg': XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=0, gpu_id=None,
              importance_type='gain', inter

In [20]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def RMSPE(y, yhat):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe 

In [21]:
#Randomized Search
parameters = {'xg__learning_rate':  uniform(0, 1),
              'xg__n_estimators':   range(50,300),
              'xg__max_depth':      range(3,10),
              'pca__n_components' : range(10,17)}

rs = RandomizedSearchCV(pipe_xg,
                        param_distributions=parameters,
                        #scoring=make_scorer(mean_squared_error, squared=False),
                        #scoring= RMSPE,
                        n_iter=15,    
                        verbose = 1,
                        #refit = RMSPE,
                        n_jobs = 3,
                        cv = 3              
                        )

In [22]:
start = time.time()
rs.fit(X_train, y_train)
end = time.time()
(end - start)/60 

Fitting 3 folds for each of 15 candidates, totalling 45 fits
[21:37:55] WARNING: ..\src\learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('sts', StandardScaler()),
                                             ('pca', PCA()),
                                             ('xg',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           gamma=0, gpu_id=None,
                                                           importance_type='gain',
                                                           interaction_constraints=None,
                                                           learning_rate=None,
                                                           max

26.87622224887212

In [23]:
# Model with parameters of random search
model_rs = XGBRegressor(objective='reg:squarederror',silent = False, n_jobs=3, reg_lambda=1,gamma=0,
                    learning_rate = rs.best_params_['xg__learning_rate'],
                    max_depth = rs.best_params_['xg__max_depth'],
                    n_estimators=rs.best_params_['xg__max_depth']
                    )


model_rs.fit(X_train, y_train)

[21:39:48] WARNING: ..\src\learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2243522518498855, max_delta_step=0, max_depth=9,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=9, n_jobs=3, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [24]:
y_pred_rs = model_rs.predict(X_test)
RMSPE(y_test,y_pred_rs)

rs.best_score_

0.2650214798730226

0.8793461001455949

In [25]:
import math
accuracy_rs =  math.sqrt(sum((y_test - y_pred_rs)**2)/y_test.count())
print("Accuracy with Random search XGB model:",accuracy_rs*100)

Accuracy with Random search XGB model: 170278.3944034436


In [26]:
X_test_df = X_test.reset_index()
y_test_df = y_test.reset_index()
y_pred_df  = pd.DataFrame(y_pred_rs)

final = X_test_df
#final
final = final.merge(y_test_df, left_index=True, right_index=True)
final = final.merge(y_pred_df, left_index=True, right_index=True)
final

,index_x,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Year,MonthNumber,Day,WeekNumber,CompetitionOpen,Promo2Open,IsPromoMonth,index_y,Sales,0
0,917117,258,7,0,0,0,0,1,1,27190,...,2013,3,31,13,32,42,0,917117,0,0.050814
1,674442,653,1,1,1,0,0,4,3,7520,...,2013,11,4,45,0,48,1,674442,7805,8112.088867
2,848032,303,6,1,0,0,0,1,1,15430,...,2013,6,1,22,7,25,0,848032,4842,4021.173584
3,435281,102,5,1,1,0,0,1,1,150,...,2014,6,6,23,78,3,1,435281,8475,6276.604004
4,587988,54,1,1,1,0,0,4,3,7170,...,2014,1,20,4,0,11,0,587988,9536,8042.581055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97293,417543,204,7,0,0,0,0,1,1,5630,...,2014,6,22,25,138,0,0,417543,0,0.050814
97294,332816,162,5,1,1,0,0,4,3,5340,...,2014,9,19,38,30,54,0,332816,8695,6934.944336
97295,483419,295,4,1,0,0,1,1,1,210,...,2014,4,24,17,161,7,0,483419,4523,4101.657227
97296,210997,263,5,1,0,0,0,1,3,1140,...,2015,1,23,4,20,3,1,210997,3511,5149.494629


In [27]:
test = pd.read_csv("test.csv")
test.head()

test.isnull().sum()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [28]:
test.Open.fillna(0, inplace= True)

In [29]:
test.isnull().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [30]:
data = pd.merge(test, ross_store,on = 'Store', how='left')
data.head()

data.shape

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN


(41088, 17)

In [31]:
ProcessData(data)

data.head()

<ipython-input-9-578d5171619f>:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['WeekNumber']=data['Date'].dt.weekofyear


,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,MonthNumber,Day,WeekNumber,CompetitionOpen,Promo2Open,IsPromoMonth
0,1,1,4,1.0,1,0,0,c,a,1270.0,0,2015,9,17,38,84.0,0.0,0
1,2,3,4,1.0,1,0,0,a,a,14130.0,1,2015,9,17,38,105.0,54.0,0
2,3,7,4,1.0,1,0,0,a,c,24000.0,0,2015,9,17,38,29.0,0.0,0
3,4,8,4,1.0,1,0,0,a,a,7520.0,0,2015,9,17,38,11.0,0.0,0
4,5,9,4,1.0,1,0,0,a,c,2030.0,0,2015,9,17,38,181.0,0.0,0


In [32]:
submission = data['Id']
data=data.drop('Id',axis=1)

In [33]:
data.head()

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,MonthNumber,Day,WeekNumber,CompetitionOpen,Promo2Open,IsPromoMonth
0,1,4,1.0,1,0,0,c,a,1270.0,0,2015,9,17,38,84.0,0.0,0
1,3,4,1.0,1,0,0,a,a,14130.0,1,2015,9,17,38,105.0,54.0,0
2,7,4,1.0,1,0,0,a,c,24000.0,0,2015,9,17,38,29.0,0.0,0
3,8,4,1.0,1,0,0,a,a,7520.0,0,2015,9,17,38,11.0,0.0,0
4,9,4,1.0,1,0,0,a,c,2030.0,0,2015,9,17,38,181.0,0.0,0


In [35]:
data['StoreType'].value_counts()
data['Assortment'].value_counts()
data['StoreType']= data['StoreType'].map({'a':1, 'b' : 2, 'c': 3, 'd' : 4})
data['Assortment'] = data['Assortment'].map({'a':1, 'b' : 2, 'c': 3})

a    22128
d    14112
c     4272
b      576
Name: StoreType, dtype: int64

c    20352
a    20304
b      432
Name: Assortment, dtype: int64

In [36]:
data = data.astype('int32')

In [37]:
num_columns = data.columns[data.nunique() > 12]
cat_columns = data.columns[data.nunique() <= 12]

In [38]:
num_columns
cat_columns

Index(['Store', 'CompetitionDistance', 'Day', 'CompetitionOpen', 'Promo2Open'], dtype='object')

Index(['DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'Promo2', 'Year', 'MonthNumber',
       'WeekNumber', 'IsPromoMonth'],
      dtype='object')

In [39]:
ct=ColumnTransformer([
    ('rs',RobustScaler(),num_columns),
    ('ohe',OneHotEncoder(),cat_columns),
    ],
    remainder="passthrough"
    )
ct.fit_transform(data)

array([[-1.        , -0.20647419,  0.36      , ...,  1.        ,
         1.        ,  0.        ],
       [-0.99638009,  2.04374453,  0.36      , ...,  1.        ,
         1.        ,  0.        ],
       [-0.98914027,  3.77077865,  0.36      , ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [ 1.01266968,  1.19160105, -0.92      , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.01447964, -0.27646544, -0.92      , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.01628959,  0.50743657, -0.92      , ...,  0.        ,
         1.        ,  0.        ]])

In [40]:
y_pred_rs = model_rs.predict(data)

In [48]:
final = submission.reset_index()
y_pred_df  = pd.DataFrame(y_pred_rs)

final = final.merge(y_pred_df, left_index=True, right_index=True)
final = final.set_axis([*final.columns[:-1], 'Predicted Sales'], axis=1, inplace=False)
final

,index,Id,Predicted Sales
0,0,1,6183.351562
1,1,2,7309.368164
2,2,3,6570.226562
3,3,4,5921.768555
4,4,5,7008.372070
...,...,...,...
41083,41083,41084,3651.837402
41084,41084,41085,4522.457520
41085,41085,41086,3904.215576
41086,41086,41087,3768.196777
